In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

In [ ]:
!pip install htmlmin
!pip install openai
!pip install html2text
!pip install python-dotenv

In [4]:
# Import necessary libraries and modules.
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import requests

# Configure Selenium WebDriver to use Chrome in a headless mode (without opening a UI window).
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [5]:
# Define a function to scrape data from a web page using Selenium and BeautifulSoup.
def get_programs(driver):
    # Navigate to the specified URL.
    driver.get("https://www.canadavisa.com/canada-pnp-finder-and-tracker.html")
    page_source=""
    try:
        # Wait until a specific element is present on the page before proceeding.
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ui celled fixed sortable table"))
        )
    finally:
        # Once the element is loaded, get the page source
        page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    table=soup.find_all("table","ui celled fixed sortable table")[0]
    headers = [th.get_text() for th in table.find('tr').find_all('th')]
    data = []

    # Extract table data into a DataFrame.
    for row in table.find_all('tr')[1:]:
        data.append([td.get_text() for td in row.find_all('td')])
    df = pd.DataFrame(data, columns=headers)

    # Extract and add hyperlink information for specific table cells.
    for i, row in enumerate(table.find_all('tr')[1:]):
        row_data = []
        for idx, td in enumerate(row.find_all('td')):
            if idx == 1:
                link = td.find('a')
                if link:
                    df.loc[i, 'Stream Link'] = link['href']
    return df.to_dict(orient='records')


In [6]:
# Define the URL and headers for making an HTTP request.
url = "https://evaluator.canadavisa.com/api/immigration/streams?lang=en"
headers = {
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Origin": "https://www.canadavisa.com",
    "Referer": "https://www.canadavisa.com/canada-pnp-finder-and-tracker.html",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "sec-ch-ua": "\"Not/A)Brand\";v=\"8\", \"Chromium\";v=\"126\", \"Google Chrome\";v=\"126\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\""
}

# Send a GET request and parse the JSON response.
response = requests.get(url, headers=headers)
data = response.json()

# Define a function to process and format JSON data into a DataFrame.
def process_streams(data):
    rows = []
    for stream in data['streams']:
        row = {
            "Province": stream["province"],
            "Category / Stream": stream["name"],
            "Program Status": stream["status"],
            "Express Entry-Linked": "Yes" if stream["express_entry"] == 1 else "No",
            "Job Required": "Yes" if stream["job_offer_required"] == 1 else "No",
            "Stream Link": stream["url"] if stream["url"] else "N/A"
        }
        rows.append(row)
    return pd.DataFrame(rows)

# Process the data and save it to a CSV file.
df = process_streams(data)
programs=df.to_dict(orient='records')
pd.DataFrame(programs).to_csv('all_programs.csv')

In [7]:
from urllib.parse import urlparse

def uri_validator(x):
    # This function checks if a given URI is valid by ensuring it has both a scheme and a network location.
    try:
        result = urlparse(x)
        return all([result.scheme, result.netloc])
    except AttributeError:
        return False

In [ ]:
# Import libraries for handling environment variables and API client setup.
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from a .env file.
api_key = os.getenv('API_KEY')  # Retrieve the API key from environment variables.
client = OpenAI(api_key=api_key)  # Initialize the OpenAI client with the API key.

In [10]:
# Import modules for handling exceptions in Selenium and for parsing HTML to text.
from selenium.common.exceptions import WebDriverException
import time
import html2text

def get_program_info(index, driver, program):
    url = program['Stream Link']  # Extract the URL for the program.
    province = program['Province']  # Extract the province information from the program data.

    # Validate the URL; if invalid, return an empty list.
    if not uri_validator(url):
        return []

    # Setup retry logic for web scraping.
    retries = 5
    delay = 2
    soup = None

    # Attempt to retrieve and parse the page content with retries on failure.
    for i in range(retries):
        try:
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            break  # Break the loop if the page is successfully retrieved and parsed.
        except WebDriverException as e:
            if i == retries - 1:
                print(f"Failed to retrieve page after {retries} attempts: {e}")
                return []
            time.sleep(delay)  # Wait before retrying, with an increasing delay.
            delay *= 2

    # Check if the page content was successfully retrieved; if not, return an empty list.
    if soup is None:
        return []

    # Attempt to find an 'article' tag in the page content.
    article = soup.find('article')
    if not article:
        print("Article not found in HTML")
        return []

    # Check the length of the article text and convert HTML to plain text if it's too long.
    size = len(article.text.split())
    if size > 16000:
        try:
            article = html2text.html2text(article)
        except:
            print(url + 'a lot ...')

    # Prepare a prompt for the OpenAI API to extract specific information from the article content.
    prompt = (
        "Extract the following information from the provided HTML content and make sure you give them as plain text, not markdown:\n"
        "1. Stream_name\n"
        "2. Substream_name (name of the sub-categories as specified in the stream)\n"
        "3. Education (type and note, type can be degrees such as elementary, secondary, and post secondary)\n"
        "4. Work_experience (job category name, NOC ID, note, and work experience length)\n"
        "5. Language_proficiency (level requirement and note)\n"
        "6. Province\n"
        "7. Last_updated\n\n"
        "If any specific information is not found, it should be labeled as Not Specified.\n\n"
        "Format the output as a JSON array with objects containing the above fields. Ensure the JSON is properly formatted and each object is correctly"
        "closed with curly braces. The array should also be enclosed with square brackets.\n\n"
        "Examples:\n\n"
        "Example 1:\n"
        "{\n"
        "  \"Stream_name\": \"Express Entry\",\n"
        "  \"Substream_name\": \"Skilled Worker\",\n"
        "  \"Education\": {\n"
        "    \"Education_type\": \"Post secondary\",\n"
        "    \"Education_note\": \"The Skilled Worker category is for international skilled workers who have post-secondary education or training.\"\n"
        "  },\n"
        "  \"Work_experience\": {\n"
        "    \"Job_category_name\": \"Professional, management, technical, trade or other skilled occupation\",\n"
        "    \"Job_NOC_id\": \"NOC 0, A, or B\",\n"
        "    \"Job_note\": \"Candidates must have a full-time permanent qualifying job offer in a skilled occupation from a B.C. employer.\",\n"
        "    \"Work_experience_length\": \"2 years\"\n"
        "  },\n"
        "  \"Language_proficiency\": {\n"
        "    \"Language_level_requirement\": \"CLB 7 or higher\",\n"
        "    \"Language_proficiency_note\": \"Candidates must meet a mandatory language proficiency level.\"\n"
        "  }\n"
        "}\n\n"
        "Example 2:\n"
        "{\n"
        "  \"Stream_name\": \"Express Entry\",\n"
        "  \"Substream_name\": \"International Graduate\",\n"
        "  \"Education\": {\n"
        "    \"Education_type\": \"secondary\",\n"
        "    \"Education_note\": \"Candidates must have graduated from a Canadian highschool or college in the last three years.\"\n"
        "  },\n"
        "  \"Work_experience\": {\n"
        "    \"Job_category_name\": \"management, statistics \",\n"
        "    \"Job_NOC_id\": \"NOC 0, 2,3\",\n"
        "    \"Job_note\": \"Candidates must have a full-time permanent job offer from a B.C. employer.\",\n"
        "    \"Work_experience_length\": \"1 year\"\n"
        "  },\n"
        "  \"Language_proficiency\": {\n"
        "    \"Language_level_requirement\": \"IELTS 5 or higher\",\n"
        "    \"Language_proficiency_note\": \"Candidates must meet a mandatory language proficiency level.\"\n"
        "  }\n"
        "}\n\n"
        "program:\n"
        f"{program}\n"
        "HTML content:\n"
        f"{article.text}\n"
    )

    try:
        # Send the prompt to the OpenAI API and capture the response.
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
    except:
        print("failure")
        return []

    # Return the processed information from the response.
    return response.choices[0].message.content

In [97]:
import numpy as np

# Define a function to check if a given value is considered safe (i.e., not None, empty, or NaN).
def is_safe(x):
    try:
        if x in [None, ''] or pd.isna(x):
            return False
    except:
        return True
    return True

# Define a function to attempt to load a string as JSON, catching exceptions and returning the original string on failure.
def json_loads(x):

     try:
        result=json.loads(x)
     except  Exception as e:
        result=x
        print(e)
     return result

# Define a function that first checks if a value is safe, then tries to load it as JSON, attempting a fix on failed JSON strings.
def safe_json_loads(x):
    if not is_safe(x):
        return []
    try:
        result=json.loads(x)
    except Exception as e:
        # Attempt to fix the JSON string by replacing problematic characters and retry loading.
        result=json_loads(str(x).replace("'","\"").replace("None","null").replace("\"s ","'s "))
    return result

In [ ]:
data = pd.DataFrame()  # Initialize an empty DataFrame to store the results.
strings = []  # List to store programs that failed to return JSON strings.
failed = []  # List to store programs for which JSON parsing failed.

# Main loop that processes each program.
for program in programs:
    json_string=get_program_info(0, driver, program)
    if len(json_string):
        try:
            # Clean up common JSON formatting issues in the string.
            json_string = json_string.replace("json\n", "").replace("`", "").strip()
            new_data = safe_json_loads(json_string)  # Attempt to load cleaned JSON string.
            if not len(new_data):
                continue

            print(program['Category / Stream'], ' Done!')
            try:
                new_df = pd.DataFrame(new_data)  # Attempt to convert the list of dictionaries to a DataFrame.
            except:
                failed.append(program)
                print("Failed!")
            new_df.dropna(how='all', inplace=True)  # Remove any rows that are entirely NaN.
            new_df['stream'] = program['Category / Stream']  # Add stream category information.
            new_df['link'] = program['Stream Link']  # Add stream link.
            data = pd.concat([data, new_df], ignore_index=True)  # Append the new DataFrame to the main DataFrame.
            print("JSON loaded successfully!")
        except json.JSONDecodeError as e:
            # Handle JSON decoding errors and print diagnostic information.
            print(f"JSONDecodeError: {e.msg}")
            print(f"Error at line {e.lineno}, column {e.colno}, char {e.pos}")
    else:
        strings.append(program)  # If no JSON string was returned, add the program to the list.

print(data)  # Display the final DataFrame.

In [ ]:
print("Number of programs failed:", len(failed))
data.to_csv('results.csv')

In [102]:
data = pd.read_csv('results.csv')

In [ ]:
import re

# Initialize an empty list to store processed records
ls = []

# Iterate through each row in the data DataFrame
for idx, pr in data.iterrows():
    # Parse the 'Education' field using a custom function that handles JSON safely
    edu = safe_json_loads(pr['Education'])
    # Extract and set the 'Education_type' and 'Education_note' fields from the parsed JSON
    pr['Education_type'] = edu.get('Education_type')
    pr['Education_note'] = edu.get('Education_note')

    # Parse the 'Work_experience' field using a custom function that handles JSON safely
    work = safe_json_loads(pr['Work_experience'])
    # Extract and set various fields related to work experience from the parsed JSON
    pr['Job_category_name'] = work.get('Job_category_name')
    pr['Job_NOC_id'] = work.get('Job_NOC_id')
    pr['Job_note'] = work.get('Job_note')
    pr['Work_experience_length'] = work.get('Work_experience_length')

    # Parse the 'Language_proficiency' field using a custom function that handles JSON safely
    lang = safe_json_loads(pr['Language_proficiency'])
    # Extract and set fields related to language proficiency from the parsed JSON
    pr['Language_level_requirement'] = lang.get('Language_level_requirement')
    pr['Language_proficiency_note'] = lang.get('Language_proficiency_note')

    # Append the modified row to the list
    ls.append(pr)

# Convert the list of dictionaries into a DataFrame specifying the columns explicitly
df = pd.DataFrame(ls, columns=[
    'Stream_name', 'Substream_name', 'Education', 'Work_experience',
    'Language_proficiency', 'Language_proficiency', 'Province', 'Last_updated',
    'stream', 'link', 'Education_type', 'Education_note', 'Job_category_name',
    'Job_NOC_id', 'Job_note', 'Work_experience_length',
    'Language_level_requirement', 'Language_proficiency_note'])

# Print the resulting DataFrame
print(df)